# 6-7. 프로젝트: 멋진 작사가 만들기
## Step 1. 데이터 다운로드
---
먼저 아래 링크에서 Song Lyrics 데이터를 다운로드하고 저장된 파일을 압축 해제한 후, 모든 txt 파일을 lyrics 폴더를 만들어 그 속에 저장하기.

* [Song Lyrics](https://www.kaggle.com/paultimothymooney/poetry/data)

아래의 명령어를 실행해도 된다.
```
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
```

## Step 2. 데이터 읽어오기
---
`glob` 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이하다. `glob` 를 활용하여 모든 `txt` 파일을 읽어온 후, `raw_corpus` 리스트에 문장 단위로 저장!

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import glob
import os


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I bought my first key from my baby momma brother', 'I bought my first key', 'Bought my bought my first key']


## Step 3. 데이터 정제
---

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

I bought my first key from my baby momma brother
I bought my first key
Bought my bought my first key
I bought my first key from my baby momma brother
I bought my first key
Bought my bought my first key
Yeah hustling on my city streets
Trying to get a whole key i bought my frist key my first key
Inand we was getting em like for twenty five
Colombian connect homey we was getting fly


`preprocess_sentence()` 함수를 사용해서 데이터 정제하기.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence.split()) <= 15:
        corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> i bought my first key from my baby momma brother <end>',
 '<start> i bought my first key <end>',
 '<start> bought my bought my first key <end>',
 '<start> i bought my first key from my baby momma brother <end>',
 '<start> i bought my first key <end>',
 '<start> bought my bought my first key <end>',
 '<start> yeah hustling on my city streets <end>',
 '<start> trying to get a whole key i bought my frist key my first key <end>',
 '<start> inand we was getting em like for twenty five <end>',
 '<start> colombian connect homey we was getting fly <end>']

## Step 4. 평가 데이터셋 분리
---
`tokenize()` 함수로 데이터를 Tensor로 변환한 후, `sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하자. 단어장의 크기는 12,000 이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로 사용하기!

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   5 613 ...   0   0   0]
 [  2   5 613 ...   0   0   0]
 [  2 613  13 ...   0   0   0]
 ...
 [  2   5  22 ...   0   0   0]
 [  2   5  22 ...   0   0   0]
 [  2   5  22 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f9700e23710>


단어사전 확인

In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


사전에는 없지만, 텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워넣은 것이다.

In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2   5 613  13 251 747  74  13  51 730 557   3   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  5 613  13 251 747  74  13  51 730 557   3   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


corpus 내의 첫번째 문장에 대해 생성된 소스와 타겟 문장을 확인해 보았다. 소스는 2(`<start>`)에서 시작해서 3(`<end>`)으로 끝난 후 0(`<pad>`)로 채워져 있다. 반면에 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한칸 시프트한 형태를 가지고 있다.

`sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리해보자.

In [8]:
from sklearn.model_selection import train_test_split

# train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,test_size=0.2,shuffle=True)


잘 분리되었는지 확인해보자.

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Train:", enc_val.shape)
print("Target Train:", dec_val.shape)

Source Train: (134685, 32)
Target Train: (134685, 32)
Source Train: (33672, 32)
Target Train: (33672, 32)


이미 데이터셋을 텐서 형태로 생성해 두었으므로, `tf.data.Dataset.from_tensor_slices()` 메소드를 이용해 `tf.data.Dataset`객체를 생성보겠다.

In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개


train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

train_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [11]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기
---
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하자!<br>
(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

```
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
```

모델은 tf.keras.Model을 Subclassing하는 방식으로 만들 것이다. 모델은 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성된다.

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in val_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-1.31574212e-04, -5.92743709e-05,  4.28307358e-05, ...,
         -1.53623128e-04, -1.16963471e-04, -3.54338022e-06],
        [-3.41774401e-04, -2.48245487e-04,  5.42396219e-06, ...,
         -1.89827217e-04, -9.76208103e-05,  3.22780630e-04],
        [-4.83807846e-04, -1.94802182e-04,  5.74491969e-05, ...,
         -2.02980242e-04, -1.95241344e-04,  2.57759937e-04],
        ...,
        [ 9.62964623e-05, -3.07124713e-03,  3.28716094e-04, ...,
          4.89815662e-04,  2.67816847e-03,  1.06990617e-03],
        [ 1.14290517e-04, -3.06320586e-03,  2.99625652e-04, ...,
          5.09878970e-04,  2.71984166e-03,  1.04257464e-03],
        [ 1.29048407e-04, -3.05549148e-03,  2.73137091e-04, ...,
          5.27583121e-04,  2.75675976e-03,  1.01917994e-03]],

       [[-1.31574212e-04, -5.92743709e-05,  4.28307358e-05, ...,
         -1.53623128e-04, -1.16963471e-04, -3.54338022e-06],
        [ 6.42408486e-06, -3.10241376e-05,  2

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
526/526 [==============================] - 155s 294ms/step - loss: 1.7052 - val_loss: 1.4714
Epoch 2/10
526/526 [==============================] - 163s 310ms/step - loss: 1.4158 - val_loss: 1.3882
Epoch 3/10
526/526 [==============================] - 157s 299ms/step - loss: 1.3432 - val_loss: 1.3373
Epoch 4/10
526/526 [==============================] - 159s 303ms/step - loss: 1.2851 - val_loss: 1.2994
Epoch 5/10
526/526 [==============================] - 159s 302ms/step - loss: 1.2341 - val_loss: 1.2661
Epoch 6/10
526/526 [==============================] - 160s 304ms/step - loss: 1.1878 - val_loss: 1.2405
Epoch 7/10
526/526 [==============================] - 162s 308ms/step - loss: 1.1456 - val_loss: 1.2186
Epoch 8/10
526/526 [==============================] - 162s 308ms/step - loss: 1.1062 - val_loss: 1.2003
Epoch 9/10
526/526 [==============================] - 166s 316ms/step - loss: 1.0698 - val_loss: 1.1845
Epoch 10/10
526/526 [==============================] - 169s 321m

In [16]:
#Loss

# loss = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

In [17]:
def generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                      tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [18]:
print(generate_text(model, tokenizer, init_sentence="<start> he"))

print(generate_text(model, tokenizer, init_sentence="<start> I love"))

print(generate_text(model, tokenizer, init_sentence="<start> My name"))

print(generate_text(model, tokenizer, init_sentence="<start> I hate"))

<start> he s a <unk> , and she s a <unk> <end> 
<start> i love you , i love you <end> 
<start> my name is prince <end> 
<start> i hate to see you <end> 


### embedding_size = 512 / hidden_size = 1024

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [21]:
for src_sample, tgt_sample in val_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-2.1527823e-04, -1.1601710e-05, -3.8312809e-04, ...,
          8.5193838e-05,  4.2805533e-04,  5.7565390e-05],
        [-4.8929045e-04,  2.9334857e-05, -6.1145495e-04, ...,
          1.2554229e-06,  6.7871076e-04, -3.1141451e-04],
        [-2.6729330e-04,  1.6775583e-04, -7.2619913e-04, ...,
         -3.6398816e-04,  9.4120624e-04, -5.0453946e-04],
        ...,
        [ 1.6228271e-03,  2.0656234e-03,  3.5932369e-03, ...,
          5.8471859e-03, -3.7177503e-03, -6.2497118e-03],
        [ 1.5848646e-03,  2.1218683e-03,  3.5528413e-03, ...,
          5.9277397e-03, -3.7573285e-03, -6.2721549e-03],
        [ 1.5493408e-03,  2.1746652e-03,  3.5153076e-03, ...,
          5.9981872e-03, -3.7892971e-03, -6.2890016e-03]],

       [[-2.1527823e-04, -1.1601710e-05, -3.8312809e-04, ...,
          8.5193838e-05,  4.2805533e-04,  5.7565390e-05],
        [-2.1442720e-04, -3.1306761e-05, -6.4488052e-04, ...,
          3.1883875e-04, 

In [22]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [23]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
526/526 [==============================] - 183s 348ms/step - loss: 1.6743 - val_loss: 1.4500
Epoch 2/10
526/526 [==============================] - 490s 932ms/step - loss: 1.3854 - val_loss: 1.3553
Epoch 3/10
526/526 [==============================] - 230s 438ms/step - loss: 1.3035 - val_loss: 1.3023
Epoch 4/10
526/526 [==============================] - 169s 322ms/step - loss: 1.2378 - val_loss: 1.2578
Epoch 5/10
526/526 [==============================] - 167s 317ms/step - loss: 1.1802 - val_loss: 1.2242
Epoch 6/10
526/526 [==============================] - 167s 317ms/step - loss: 1.1275 - val_loss: 1.1970
Epoch 7/10
526/526 [==============================] - 176s 334ms/step - loss: 1.0790 - val_loss: 1.1743
Epoch 8/10
526/526 [==============================] - 175s 333ms/step - loss: 1.0339 - val_loss: 1.1569
Epoch 9/10
526/526 [==============================] - 175s 333ms/step - loss: 0.9914 - val_loss: 1.1409
Epoch 10/10
526/526 [==============================] - 174s 332m

In [24]:
def generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                      tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
print(generate_text(model, tokenizer, init_sentence="<start> he"))

print(generate_text(model, tokenizer, init_sentence="<start> I love"))

print(generate_text(model, tokenizer, init_sentence="<start> My name"))

print(generate_text(model, tokenizer, init_sentence="<start> I hate"))

<start> he s got a couple of animals <end> 
<start> i love you , i love you <end> 
<start> my name is prince <end> 
<start> i hate to see you smile <end> 


### embedding_size = 256 / hidden_size = 2048

In [26]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [27]:
for src_sample, tgt_sample in val_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-3.96370388e-06,  2.18157420e-05,  2.11191262e-04, ...,
         -2.07480662e-06,  5.93301920e-05,  1.07609048e-04],
        [ 1.82150397e-05, -4.40818621e-06,  3.74529947e-04, ...,
         -6.47407505e-05, -3.79609010e-05,  1.23202248e-04],
        [-1.73640947e-04, -5.62932910e-05,  3.00176558e-04, ...,
          1.30866683e-04, -3.02157889e-04,  3.36859492e-04],
        ...,
        [ 1.92237785e-03,  2.85950303e-03, -4.06129472e-03, ...,
          4.62022232e-04,  1.02168613e-03,  1.16832372e-04],
        [ 1.91388722e-03,  2.88365781e-03, -4.11089649e-03, ...,
          4.94447653e-04,  1.02099427e-03,  1.20408287e-04],
        [ 1.90602208e-03,  2.90388078e-03, -4.15120507e-03, ...,
          5.22487331e-04,  1.01961556e-03,  1.23087273e-04]],

       [[-3.96370388e-06,  2.18157420e-05,  2.11191262e-04, ...,
         -2.07480662e-06,  5.93301920e-05,  1.07609048e-04],
        [ 1.24002894e-04,  2.49934150e-04,  3

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

In [ ]:
def generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                      tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
print(generate_text(model, tokenizer, init_sentence="<start> he"))

print(generate_text(model, tokenizer, init_sentence="<start> I love"))

print(generate_text(model, tokenizer, init_sentence="<start> My name"))

print(generate_text(model, tokenizer, init_sentence="<start> I hate"))

## 루브릭

__1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?__<br>
텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
* 가사 텍스트 생성 모델이 정상적으로 작동하고, 그럴듯한 문장이 생성되었다.

__2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?__<br>
특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
* 특수문자 제거, 토크나이저 생성, 패딩처리 등 과정이 빠짐없이 진행되었다. 단, 패딩처리가 좀 과하게 된 느낌이다. 해결하려고 해도 해결되지 않았다.

__3. 텍스트 생성모델이 안정적으로 학습되었는가?__<br>
텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?
* 잘은 모르겠지만 처음부터 loss가 2.2보다 낮았다. 솔직히 좀 모르겠다.

## 느낀점

그냥 겁나게 어려웠다. 자연어는 어렵다는 인식이 점점...ㅋㅋ 강해지는거 같다. 이해도 어렵고 생각도 못하고~ㅋㅋㅋ 그래도 그냥 버티고 있다...